# US_Travel_Flows

### Objectives:
Create pivot tables to summarize travel flows among states or HHS regions. Each travel mode (driving, public transit and other) has its separate table.

NetworkX?

In [80]:
import pandas as pd
folder = r'C:\Users\Ensheng\Desktop\Flu_B\\'
out_folder = r'C:\Users\Ensheng\Desktop\Flu_B\outputs\\'

## Commuting Flows

In [81]:
in_table = folder + r'commuting_flows_by_travel_mode.csv'
df_com = pd.read_csv(in_table)
df_com.head(3)

,res_state_FIPS,res_county_FIPS,res_FIPS,res_state,res_county,work_state_FIPS,work_county_FIPS,work_FIPS,work_state,work_county,travel_mode,flow,moe
0,1,1,1001,Alabama,Autauga County,1.0,1.0,1001.0,Alabama,Autauga County,driving,6963,532
1,1,1,1001,Alabama,Autauga County,1.0,1.0,1001.0,Alabama,Autauga County,driving,867,201
2,1,1,1001,Alabama,Autauga County,1.0,1.0,1001.0,Alabama,Autauga County,public_transit,9,10


In [82]:
# remove unrelated residence/work locations
df_com = df_com[df_com['work_state']!='Other workplace location outside United States'].reset_index(drop=True)
df_com = df_com[df_com['work_state']!='Canada'].reset_index(drop=True) # FIPS = 301
df_com = df_com[df_com['work_state']!='Mexico'].reset_index(drop=True) # FIPS = 303

In [83]:
set(df_com['res_FIPS']) - set(df_com['work_FIPS'])

set()

In [84]:
set(df_com['work_FIPS']) - set(df_com['res_FIPS'])

set()

In [85]:
in_table = folder + r'HHS_states_table.csv'
df_hhs = pd.read_csv(in_table)
df_hhs.head(3)

,HHS,Regions,state_FIPS
0,HHS04,Alabama,1
1,HHS10,Alaska,2
2,HHS09,Arizona,4


In [86]:
# merge with HHS
df_temp = pd.merge(df_com, df_hhs, how='left', left_on='res_state_FIPS',right_on='state_FIPS')
df_temp = pd.merge(df_temp, df_hhs, how='left', left_on='work_state_FIPS',right_on='state_FIPS')
df_temp = df_temp.rename(index=str, columns={"HHS_x": "res_HHS", "HHS_y": "work_HHS"})
df_temp = df_temp.drop(columns=['Regions_x', 'state_FIPS_x', 'Regions_y', 'state_FIPS_y'])
df_com = df_temp

In [87]:
# separate into different transportation modes and check its availabity
df_driving = df_com[df_com['travel_mode']=='driving'].reset_index(drop=True)
df_public = df_com[df_com['travel_mode']=='public_transit'].reset_index(drop=True)
df_other = df_com[df_com['travel_mode']=='other_mode'].reset_index(drop=True)

In [88]:
print(len(set(df_com['res_FIPS'])))
print(len(set(df_com['work_FIPS'])))
print(len(set(df_driving['res_FIPS'])))
print(len(set(df_driving['work_FIPS'])))
print(len(set(df_public['res_FIPS'])))
print(len(set(df_public['work_FIPS'])))
print(len(set(df_other['res_FIPS'])))
print(len(set(df_other['work_FIPS'])))
# note: the smaller counts mean incomplete number of counties for poblic transit and other modes.

3221
3221
3221
3221
2728
2642
3221
3221


In [141]:
print(len(set(df_driving['res_FIPS']) - set(df_driving['work_FIPS'])))
print(len(set(df_driving['work_FIPS']) - set(df_driving['res_FIPS'])))
print(len(set(df_public['res_FIPS']) - set(df_public['work_FIPS'])))
print(len(set(df_public['work_FIPS']) - set(df_public['res_FIPS'])))
print(len(set(df_other['res_FIPS']) - set(df_other['work_FIPS'])))
print(len(set(df_other['work_FIPS']) - set(df_other['res_FIPS'])))

0
0
149
63
0
0


### Pivoting tables and sorting flow

In [132]:
mode = 'other'

if mode == 'driving':
    df_mode = df_driving
if mode == 'public':
    df_mode = df_public
if mode == 'other':
    df_mode = df_other

output_name_state = 'Flows_' + mode + '_state.csv'
output_name_state_rank = 'Flows_' + mode + '_state_rank.csv'
output_name_hhs = 'Flows_' + mode + '_HHS.csv'
output_name_hhs_rank = 'Flows_' + mode + '_HHS_rank.csv'

### State summary

In [133]:
result_temp = df_mode.groupby(['res_state', 'work_state'])['flow'].sum()
result = result_temp.unstack().fillna(0)
output_csv = out_folder + output_name_state
result.to_csv(output_csv, index=True, encoding='utf-8')
result

work_state,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
res_state,,,,,,,,,,,,,,,,,,,,,
Alabama,90626.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,18.0,129.0,...,0.0,96.0,197.0,0.0,0.0,18.0,0.0,0.0,4.0,0.0
Alaska,0.0,62029.0,0.0,0.0,0.0,4.0,0.0,0.0,38.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0
Arizona,14.0,113.0,269415.0,0.0,1304.0,160.0,0.0,0.0,42.0,100.0,...,0.0,90.0,238.0,207.0,0.0,14.0,61.0,0.0,59.0,0.0
Arkansas,0.0,0.0,0.0,74295.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,107.0,282.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
California,0.0,6.0,856.0,0.0,1665450.0,140.0,22.0,0.0,289.0,213.0,...,0.0,32.0,1357.0,117.0,0.0,136.0,1006.0,0.0,13.0,0.0
Colorado,0.0,0.0,299.0,0.0,657.0,293383.0,0.0,0.0,12.0,2.0,...,0.0,6.0,446.0,67.0,0.0,7.0,61.0,0.0,0.0,23.0
Connecticut,0.0,0.0,0.0,0.0,66.0,0.0,138808.0,0.0,40.0,23.0,...,0.0,0.0,80.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0
Delaware,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28826.0,75.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0
District of Columbia,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,64236.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1499.0,10.0,0.0,0.0,0.0


In [134]:
# sort by flow where res and work are different locations
result_flow = result_temp.reset_index()
result_flow = result_flow[result_flow['res_state'] != result_flow['work_state']]
df_flow = result_flow.sort_values('flow',ascending = False).reset_index(drop=True)

result = df_flow
output_csv = out_folder + output_name_state_rank
result.to_csv(output_csv, index=True, encoding='utf-8')

df_flow.head(10)

,res_state,work_state,flow
0,New Jersey,New York,5051
1,Virginia,District of Columbia,4934
2,Maryland,District of Columbia,3573
3,New York,New Jersey,2924
4,New Jersey,Pennsylvania,1703
5,Pennsylvania,New Jersey,1630
6,District of Columbia,Virginia,1499
7,New York,Connecticut,1449
8,Florida,New York,1440
9,Florida,Georgia,1393


### HHS summary

In [135]:
# output pivot table
result_temp = df_mode.groupby(['res_HHS', 'work_HHS'])['flow'].sum()
result = result_temp.unstack().fillna(0)
output_csv = out_folder + output_name_hhs
result.to_csv(output_csv, index=True, encoding='utf-8')
result

work_HHS,HHS01,HHS02,HHS03,HHS04,HHS05,HHS06,HHS07,HHS08,HHS09,HHS10
res_HHS,,,,,,,,,,
HHS01,697638.0,2803.0,764.0,272.0,466.0,161.0,0.0,0.0,258.0,4.0
HHS02,2501.0,1501553.0,3143.0,830.0,818.0,540.0,5.0,24.0,884.0,67.0
HHS03,497.0,3282.0,1194115.0,1159.0,905.0,376.0,152.0,0.0,253.0,32.0
HHS04,429.0,3174.0,2643.0,1858655.0,2946.0,3029.0,307.0,49.0,1340.0,71.0
HHS05,327.0,1155.0,1301.0,1951.0,1841324.0,669.0,1879.0,605.0,1234.0,158.0
HHS06,126.0,601.0,375.0,1750.0,1269.0,1212908.0,259.0,410.0,1353.0,137.0
HHS07,33.0,40.0,85.0,111.0,1229.0,474.0,533708.0,164.0,58.0,7.0
HHS08,20.0,243.0,103.0,168.0,514.0,580.0,171.0,587059.0,1711.0,303.0
HHS09,201.0,1766.0,1031.0,856.0,2125.0,2059.0,224.0,835.0,2108573.0,1527.0


In [136]:
# sort by flow where res and work are different locations
result_flow = result_temp.reset_index()
result_flow = result_flow[result_flow['res_HHS'] != result_flow['work_HHS']]
df_flow = result_flow.sort_values('flow',ascending = False).reset_index(drop=True)

result = df_flow
output_csv = out_folder + output_name_hhs_rank
result.to_csv(output_csv, index=True, encoding='utf-8')

df_flow.head(10)

,res_HHS,work_HHS,flow
0,HHS03,HHS02,3282
1,HHS04,HHS02,3174
2,HHS02,HHS03,3143
3,HHS04,HHS06,3029
4,HHS04,HHS05,2946
5,HHS01,HHS02,2803
6,HHS04,HHS03,2643
7,HHS02,HHS01,2501
8,HHS09,HHS05,2125
9,HHS09,HHS06,2059


## Domestic Flights

In [143]:
in_table = folder + r'iata_flows.csv'
df_iata = pd.read_csv(in_table)
df_iata.head(3)

,org,dst,paxisplus
0,AAE,ALG,4780
1,AAE,HME,220
2,AAE,IST,825


In [150]:
# import airport data getting from the internet
in_table = r'C:\Users\Ensheng\Desktop\mapping\IATA\IATA_airports.xlsx'
df_airports = pd.read_excel(in_table)
print(len(df_airports))
df_airports.head(5)

2618


,Code,Name,City,State,State Name,Latitude,Longitude,Country,Country Name,Global Region
5,AAF,Apalachicola Regional Airport,Apalachicola,FL,Florida,29.73,-84.98,US,United States,North America
15,AAP,Andrau Airpark,Houston,TX,Texas,29.83,-95.67,US,United States,North America
30,ABE,Lehigh Valley International,Allentown,PA,Pennsylvania,40.65,-75.44,US,United States,North America
34,ABI,Abilene Regional Airport,Abilene,TX,Texas,32.41,-99.68,US,United States,North America
37,ABL,Ambler River,Ambler,AK,Alaska,67.09,-157.85,US,United States,North America


In [160]:
df_airports = df_airports[df_airports['Country'] == 'US']
df_airports = df_airports[['Code','State Name']]
df_airports = df_airports[df_airports['State Name']!='U.S. Pacific Trust Territories And Possessions'].reset_index(drop=True)
df_airports = df_airports[df_airports['State Name']!='U.S. Virgin Islands'].reset_index(drop=True)

In [161]:
print(len(set(df_airports['State Name']) - set(df_com['res_state'])))
print(len(set(df_com['res_state']) - set(df_airports['State Name'])))

0
0


In [163]:
print(len(df_airports))
df_airports.head(5)

2601


,Code,State Name
0,AAF,Florida
1,AAP,Texas
2,ABE,Pennsylvania
3,ABI,Texas
4,ABL,Alaska


In [169]:
# merge with HHS
df_temp = pd.merge(df_airports, df_hhs, how='left', left_on='State Name',right_on='Regions')
df_airports = df_temp
print(len(df_airports.loc[df_airports['Regions'].isnull()]))
df_airports.head(5)

0


,Code,State Name,HHS,Regions,state_FIPS
0,AAF,Florida,HHS04,Florida,12
1,AAP,Texas,HHS06,Texas,48
2,ABE,Pennsylvania,HHS03,Pennsylvania,42
3,ABI,Texas,HHS06,Texas,48
4,ABL,Alaska,HHS10,Alaska,2


In [178]:
# merge with HHS
df_temp = pd.merge(df_iata, df_airports, how='left', left_on='org',right_on='Code')
df_temp = pd.merge(df_temp, df_airports, how='left', left_on='dst',right_on='Code')
df_temp = df_temp[df_temp['Code_x'].notnull()]
df_temp = df_temp[df_temp['Code_y'].notnull()]
# mimic the field names as in the commuting section
df_temp = df_temp.rename(index=str, columns={"Regions_x": "res_state", "Regions_y": "work_state",\
                                            "HHS_x": "res_HHS", "HHS_y": "work_HHS",'paxisplus': 'flow'}) 
#df_temp = df_temp.drop(columns=['Regions_x', 'state_FIPS_x', 'Regions_y', 'state_FIPS_y'])
df_temp = df_temp[['res_state','res_HHS','work_state','work_HHS','flow']]
df_flight = df_temp
df_flight.head(10)

,res_state,res_HHS,work_state,work_HHS,flow
1177,Pennsylvania,HHS03,Georgia,HHS04,806
1178,Pennsylvania,HHS03,North Carolina,HHS04,526
1179,Pennsylvania,HHS03,Michigan,HHS05,324
1180,Pennsylvania,HHS03,Wisconsin,HHS05,7
1181,Pennsylvania,HHS03,Illinois,HHS05,401
1182,Pennsylvania,HHS03,Florida,HHS04,1212
1183,Pennsylvania,HHS03,Pennsylvania,HHS03,14
1184,Pennsylvania,HHS03,Florida,HHS04,1356
1185,Pennsylvania,HHS03,Florida,HHS04,4270
1186,Pennsylvania,HHS03,New Mexico,HHS06,40


### Pivoting tables and sorting flow (similar to the commuting flow)

In [179]:
mode = 'flight'

if mode == 'flight':
    df_mode = df_flight

output_name_state = 'Flows_' + mode + '_state.csv'
output_name_state_rank = 'Flows_' + mode + '_state_rank.csv'
output_name_hhs = 'Flows_' + mode + '_HHS.csv'
output_name_hhs_rank = 'Flows_' + mode + '_HHS_rank.csv'

### State summary

In [180]:
result_temp = df_mode.groupby(['res_state', 'work_state'])['flow'].sum()
result = result_temp.unstack().fillna(0)
output_csv = out_folder + output_name_state
result.to_csv(output_csv, index=True, encoding='utf-8')
result

work_state,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
res_state,,,,,,,,,,,,,,,,,,,,,
Alabama,57.0,421.0,3414.0,658.0,12756.0,4221.0,873.0,0.0,6935.0,17930.0,...,210.0,578.0,20571.0,1447.0,187.0,3760.0,2350.0,199.0,1052.0,280.0
Alaska,400.0,155294.0,4504.0,357.0,18989.0,4965.0,191.0,0.0,1451.0,4258.0,...,269.0,970.0,8086.0,2463.0,76.0,854.0,31198.0,57.0,686.0,207.0
Arizona,3177.0,4729.0,1297.0,2047.0,173559.0,66214.0,2723.0,0.0,9250.0,44623.0,...,5951.0,11004.0,69686.0,26569.0,616.0,8839.0,42572.0,394.0,14340.0,949.0
Arkansas,652.0,403.0,2378.0,5.0,12500.0,3260.0,435.0,0.0,2451.0,10652.0,...,132.0,1606.0,13503.0,1051.0,72.0,1902.0,2079.0,107.0,1028.0,189.0
California,12877.0,19298.0,177586.0,12380.0,986307.0,188220.0,13428.0,0.0,45147.0,214152.0,...,5278.0,46090.0,352212.0,108534.0,3368.0,92699.0,295433.0,1147.0,29275.0,5292.0
Colorado,4504.0,5625.0,71655.0,3449.0,197055.0,7082.0,5454.0,0.0,17950.0,90446.0,...,4790.0,16455.0,146161.0,24948.0,1427.0,26635.0,31873.0,528.0,16736.0,2037.0
Connecticut,1036.0,203.0,2590.0,425.0,12008.0,5686.0,0.0,0.0,9010.0,63195.0,...,204.0,3147.0,8351.0,1171.0,2.0,2453.0,1931.0,119.0,1229.0,226.0
Delaware,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3429.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
District of Columbia,6689.0,1491.0,9786.0,2445.0,46693.0,18496.0,8541.0,0.0,0.0,106630.0,...,625.0,18740.0,51558.0,6066.0,2709.0,342.0,10504.0,269.0,9057.0,443.0


In [181]:
# sort by flow where res and work are different locations
result_flow = result_temp.reset_index()
result_flow = result_flow[result_flow['res_state'] != result_flow['work_state']]
df_flow = result_flow.sort_values('flow',ascending = False).reset_index(drop=True)

result = df_flow
output_csv = out_folder + output_name_state_rank
result.to_csv(output_csv, index=True, encoding='utf-8')

df_flow.head(10)

,res_state,work_state,flow
0,Florida,New York,565582
1,New York,Florida,561772
2,California,Texas,352212
3,Texas,California,350894
4,California,Washington,295433
5,Washington,California,291852
6,Nevada,California,270215
7,Florida,Texas,264312
8,New York,California,260166
9,California,New York,258156


### HHS summary

In [182]:
# output pivot table
result_temp = df_mode.groupby(['res_HHS', 'work_HHS'])['flow'].sum()
result = result_temp.unstack().fillna(0)
output_csv = out_folder + output_name_hhs
result.to_csv(output_csv, index=True, encoding='utf-8')
result

work_HHS,HHS01,HHS02,HHS03,HHS04,HHS05,HHS06,HHS07,HHS08,HHS09,HHS10
res_HHS,,,,,,,,,,
HHS01,10892,110681,189037,434040,143885,89746,25005,48696,175689,29207
HHS02,114798,226040,129113,1220679,323523,248158,55279,94689,514080,58905
HHS03,181382,126103,39366,870707,309491,287940,77901,125850,402204,74593
HHS04,427220,1226026,882228,1133354,1106516,716465,213580,259928,720844,136836
HHS05,144951,315843,313875,1106100,297489,375024,121507,219349,739633,120674
HHS06,87763,246900,289733,697312,367772,622506,121780,253402,723280,132956
HHS07,23959,53857,79244,211418,119246,125824,18574,64215,204320,43441
HHS08,48847,96258,126213,264971,217065,261652,68062,123805,590075,133486
HHS09,182510,508398,404809,708311,712179,724633,196020,567675,2847624,746616


In [183]:
# sort by flow where res and work are different locations
result_flow = result_temp.reset_index()
result_flow = result_flow[result_flow['res_HHS'] != result_flow['work_HHS']]
df_flow = result_flow.sort_values('flow',ascending = False).reset_index(drop=True)

result = df_flow
output_csv = out_folder + output_name_hhs_rank
result.to_csv(output_csv, index=True, encoding='utf-8')

df_flow.head(10)

,res_HHS,work_HHS,flow
0,HHS04,HHS02,1226026
1,HHS02,HHS04,1220679
2,HHS04,HHS05,1106516
3,HHS05,HHS04,1106100
4,HHS04,HHS03,882228
5,HHS03,HHS04,870707
6,HHS10,HHS09,751654
7,HHS09,HHS10,746616
8,HHS05,HHS09,739633
9,HHS09,HHS06,724633
